### 0. Data Loading

프로젝트 폴더에서 데이터를 불러온다 

In [1]:
import re
import pandas as pd
import numpy as np 
import json


dataOutPath='./data_out/'
File = 'train_all.csv'
#testFile = 'FDOT-clean.csv'

Input = pd.read_csv(dataOutPath+File)
#testInput = pd.read_csv(dataOutPath+testFile)  


In [2]:
test_data = Input['clauses'][:414]
test_label = Input['label'][:414]

train_data = Input['clauses'][415:]
train_label = Input['label'][415:]

train_data_df = pd.DataFrame(data = {"clauses": train_data, "label":train_label})
train_data_df.to_csv(dataOutPath+'train_data.csv')

test_data_df = pd.DataFrame(data = {"clauses": test_data, "label":test_label})
test_data_df.to_csv(dataOutPath+'test_data.csv')

### TF-IDF vectorizing

- ngram_range: 단어 빈도의 기본단위를 얼만큼의 n-gram으로 설정할지 
- min_df : 설정한 값보다 특정 토큰의 df 값이 더 작게 나오면 벡터화에서 제거 
- analyzer: char / word - 임베딩 단위를 문자로할지 단어로할지. 
- sublinear_tf: 문서의 단어 빈도수에 대한 스무딩(smoothing)을 설정 

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=1000,
                             ngram_range=(1,1),
                            min_df=0.0,
                            analyzer="word",
                            sublinear_tf =True)

train_vec=vectorizer.fit_transform(train_data)
test_vec = vectorizer.transform(test_data)

In [4]:
train_vec

<1659x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 19401 stored elements in Compressed Sparse Row format>

### 3. 훈련데이터와 검증데이터 나누기

전체 훈련데이터에 사용되는 문장 4049개를  t_size 비율만큼 훈련:검증 데이터로 나눈다. 
sklearn의 train_test_split 함수를 사용해서 자동으로 분할한다. 

In [5]:
from sklearn.model_selection import train_test_split

t_size=0.2 
r_seed=42

train_cl, eval_cl, train_lb, eval_lb = train_test_split(train_vec, train_label, test_size=t_size, random_state=r_seed )

### 4. 랜덤포레스트 분류

- RandomForestClassifier 는 sklearn의 ensemble  라이브러리에서 제공한다. 
- forest라는 이름으로 분류기 객체를 생성하고, n_estimators 변수로 결정트리의 개수를 지정한다. 
- forest.fit 함수의 인자로 훈련 데이터와 레이블을 주입한다. 
- 검증데이터셋 (eval_cl, eval_lb)로 분류정확도를 측정한다. 


In [6]:
from sklearn.ensemble import RandomForestClassifier 

forest = RandomForestClassifier(n_estimators=100)

forest.fit(train_cl, train_lb)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [7]:
print("Accuracy of training: %f" % forest.score(train_cl, train_lb))
print("Accuracy: %f" % forest.score(eval_cl, eval_lb))

Accuracy of training: 1.000000
Accuracy: 0.915663


### 4.1 분류결과 
- n_estimators =n 일때, n 값이 커질수록 정확도가 향상됨을 알 수 있다. 
- n=100 일때 accuracy = 95% 달성 

In [8]:
import os 
# 테스트 데이터 파일에 쓰기 -

DATA_OUT_PATH='./data_out/results/'

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)
    
# 위에서 만든 랜덤 포레스트 분류기를 통해 예측값을 가져온다.
result = forest.predict(test_vec)

# 판다스 데이터 프레임을 통해 데이터를 구성해서 output에 넣는다.
output = pd.DataFrame( data={"label":test_label,  "predict": result} )

# 이제 csv파일로 만든다.
output.to_csv(DATA_OUT_PATH + "tfidf-predict-test.csv", index=False, quoting=3 )